# Uppgift 1

In [254]:
import pandas as pd
import seaborn as sns
import hashlib as hl
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("..")
from data_utils import *
import matplotlib.image as mpimg
import plotly.express as px

# FutureWarning handling
pd.options.mode.copy_on_write = True

def medal_counter(df=nor, col="Games"):
    df_medal = df.copy()
    df_medal["Medal"] = df_medal["Medal"].fillna("No Medal")
    df_medal = df_medal.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    medal_count = df_medal.groupby([col, "Medal"]).size().unstack(fill_value=0)
    medal_count["Total"] = medal_count[["Bronze", "Silver", "Gold"]].sum(axis=1)
    medal_count = medal_count.reindex(columns=["Bronze", "Silver", "Gold", "Total"])
    return medal_count


df = read_athlete_events("/home/albot/coding/repos/Projekt-OS-Norge/athlete_events.csv")
df = hash_column(df, "Name")
nor = df[df["NOC"] == "NOR"]
nor_medals = medal_counter()

In [255]:
fig = px.line(nor_medals, x=nor_medals.index, y="Total", markers=True, title="Medals won by Norway in the Olympics")
fig.show()

## Gender distribution  

### Perhaps  
* age  
* medals  
* px subplot pie chart of women w/ medals vs women participating, same for men  
* sns barplot (countplot?) of women particiapting through the years
* best sports

In [256]:
fig = px.histogram(nor, x="Age", color="Sex", barmode="overlay", title="Average age of Norwegian Olympic athletes", labels={"Age": "Age", "Sex": "Gender"})
fig.update_traces(marker_line_width=1.5)
fig.show()

In [ ]:
nor_wom = nor[nor["Sex"] == "F"]
nor_men = nor[nor["Sex"] == "M"]
nor_medals_wom = medal_counter(nor_wom, "Games").sort_values(by="Games")
nor_medals_men = medal_counter(nor_men, "Games").sort_values(by="Games")

fig = px.line(nor_medals.reset_index(), x="Games", y="Total", color_discrete_sequence=["crimson"], labels={"Total": "Overall"}, title="Norwegian Olympic medals")
fig.add_scatter(x=nor_medals_men.reset_index()["Games"], y=nor_medals_men["Total"], mode="lines", name="Men", line=dict(color="forestgreen"))
fig.add_scatter(x=nor_medals_wom.reset_index()["Games"], y=nor_medals_wom["Total"], mode="lines", name="Women", line=dict(color="orange"))
fig.update_layout(xaxis_title="Year", yaxis_title="Number of medals", legend_title_text="Category")
fig.update_xaxes(tickangle=-90)
fig.show()

# FIXME: t ex, 1920 har overall 32, men 32 och women 1. 2014 har overall 26, men 14 och women 13. 

In [288]:
nor_athletes = nor.groupby("Games")["ID"].nunique().reset_index(name="All")
nor_athletes_wom = nor_wom.groupby("Games")["ID"].nunique().reset_index(name="Women")
nor_athletes_men = nor_men.groupby("Games")["ID"].nunique().reset_index(name="Men")
nor_athletes = nor_athletes.merge(nor_athletes_wom, on="Games", how="left").fillna(0)
nor_athletes = nor_athletes.merge(nor_athletes_men, on="Games", how="left").fillna(0)
nor_athletes[["Women", "Men"]] = nor_athletes[["Women", "Men"]].astype(int)
nor_athletes.head()

,Games,All,Women,Men
0,1900 Summer,7,0,7
1,1904 Summer,3,0,3
2,1906 Summer,32,0,32
3,1908 Summer,69,0,69
4,1912 Summer,190,2,188


In [284]:
fig = px.bar(nor_athletes, x="Games", y=["Women", "Men"], 
             color_discrete_sequence=["orange", "forestgreen"], 
             title="Norwegian athletes in the Olympics", 
             labels={"value": "Pizza", "variable": "Gender", "Games": ""})
fig.update_xaxes(tickangle=-90)
fig.show()

In [306]:
nor_athletes["Women%"] = ((nor_athletes["Women"] / nor_athletes["All"]) * 100).round(0).astype(int)
nor_athletes["Men%"] = ((nor_athletes["Men"] / nor_athletes["All"]) * 100).round(0).astype(int)
nor_athletes = nor_athletes.reindex(columns=["Games", "All", "Women", "Women%", "Men", "Men%"])
display(nor_athletes.tail())
fig = px.line(nor_athletes, x="Games", y=["Women%", "Men%"], color_discrete_sequence=["orange", "forestgreen"], labels={"value": "Percentage", "variable": "Gender"}, title="Women vs men in the Norwegian Olympics teams")
fig.update_xaxes(tickangle=-90)
fig.show()

,Games,All,Women,Women%,Men,Men%
44,2008 Summer,84,54,64,30,36
45,2010 Winter,95,25,26,70,74
46,2012 Summer,61,28,46,33,54
47,2014 Winter,110,32,29,78,71
48,2016 Summer,62,33,53,29,47
